<a href="https://colab.research.google.com/github/Sidhtang/bert-project/blob/main/fast_api_for_the_mode_l.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import torch
from transformers import AutoTokenizer
import os

# Check if MPS is available and set the device accordingly
device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')
print(f"Using device: {device}")

# Load the model
model_name = "/content/drive/MyDrive/distilbert_general_router_model (1).pth"

# Check if the model file exists and is not empty
if not os.path.exists(model_name) or os.stat(model_name).st_size == 0:
    print(f"Error: Model file not found or empty: {model_name}")
else:
    # Load the tokenizer
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

    try:
        model = torch.load(model_name, map_location=device)
    except RuntimeError as e:
        print(f"Error loading model: {e}")
    else:
        model.eval()  # Put the model in evaluation mode
        model.to(device)

        # Function for inference
        def classify_query(query):
            inputs = tokenizer(query, return_tensors='pt', max_length=256, truncation=True, padding=True)
            inputs = {k: v.to(device) for k, v in inputs.items()}  # Move inputs to the correct device
            with torch.no_grad():
                outputs = model(**inputs)

            logits = outputs.logits
            prediction = torch.argmax(logits, dim=1).item()
            return "Personalization" if prediction == 1 else "Customer_support"

        # Test the model with queries
        queries = [
            "Hey there, you guys got some nice hoodies for me?"
        ]
        for query in queries:
            result = classify_query(query)
            print(f"Query: {query}")
            print(f"Prediction: {result}")
            print()

Using device: cpu


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
<ipython-input-2-09382d3db9af>:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unles

Query: Hey there, you guys got some nice hoodies for me?
Prediction: Personalization



In [3]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [5]:
!pip install fastapi

In [6]:
!pip install uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00


In [5]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.4 MB/s eta 0:00:00


In [6]:
import torch
from transformers import AutoTokenizer
import os
import gradio as gr

# Check if MPS is available and set the device accordingly
device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')
print(f"Using device: {device}")

# Load the model
model_name = "/content/drive/MyDrive/distilbert_general_router_model (1).pth"

# Check if the model file exists and is not empty
if not os.path.exists(model_name) or os.stat(model_name).st_size == 0:
    print(f"Error: Model file not found or empty: {model_name}")
else:
    # Load the tokenizer
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

    try:
        model = torch.load(model_name, map_location=device)
    except RuntimeError as e:
        print(f"Error loading model: {e}")
    else:
        model.eval()  # Put the model in evaluation mode
        model.to(device)

        # Function for inference
        def classify_query(query):
            inputs = tokenizer(query, return_tensors='pt', max_length=256, truncation=True, padding=True)
            inputs = {k: v.to(device) for k, v in inputs.items()}  # Move inputs to the correct device
            with torch.no_grad():
                outputs = model(**inputs)

            logits = outputs.logits
            prediction = torch.argmax(logits, dim=1).item()
            return "Personalization" if prediction == 1 else "Customer_support"

        # Gradio interface
        def gradio_interface(query):
            result = classify_query(query)
            return result

        # Create and launch the Gradio interface
        iface = gr.Interface(
            fn=gradio_interface,
            inputs=gr.Textbox(lines=2, placeholder="Enter your query here..."),
            outputs="text",
            title="Query Classifier",
            description="Classify queries as 'Personalization' or 'Customer_support'",
        )

        iface.launch()

Using device: cpu


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
<ipython-input-6-27b0428c6c73>:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unles

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://198d31f68cd72f0257.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
